In [5]:
import os
import json
import sqlite3

In [6]:
def load_all_votes() -> list[dict[str, str]]:
    # Get all files in ../votes
    votes = []
    for file in os.listdir("../votes"):
        if file.endswith(".json"):
            with open(f"../votes/{file}", "r") as f:
                votes.append(json.load(f))
    return votes

In [ ]:
def generate_db(votes: list[dict[str, str]]) -> None:
    # create a new file in ../db/votes.sqlite

    if not os.path.exists("../db"):
        os.makedirs("../db")

    if not os.path.exists("../db/votes.sqlite"):
        open("../db/votes.sqlite", "w").close()

    conn = sqlite3.connect("../db/votes.sqlite")
    cursor = conn.cursor()

    # create a table called votes
    cursor.execute(
        "CREATE TABLE IF NOT EXISTS votes (id INTEGER PRIMARY KEY, title TEXT, subtitle TEXT, detail_text TEXT, short_description TEXT, submitting_party TEXT, vote_category TEXT)"
    )

    # Convert dictionary values to tuples in the correct order
    vote_tuples = [
        (
            vote.get("id"),
            vote.get("title"),
            vote.get("subtitle"),
            vote.get("detail_text"),
            vote.get("short_description"),
            ", ".join(sorted(vote.get("submitting_parties", []))),
            vote.get("vote_category"),
        )
        for vote in votes
    ]

    print(vote_tuples)
    # insert all votes into the table
    cursor.executemany(
        "INSERT INTO votes (id, title, subtitle, detail_text, short_description, submitting_party, vote_category) VALUES (?, ?, ?, ?, ?, ?, ?)",
        vote_tuples,
    )

    conn.commit()
    conn.close()


generate_db(load_all_votes())